In [ ]:
!pip install ultralytics==8.0.208

In [ ]:
import zipfile
import os
import shutil
from ultralytics          import YOLO
from google.colab         import drive
import random
import tqdm

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


## YOLO_DATASET SHUFFLE + COCO

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/yolo_dataset.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
img_dir = '/content/coco-images'
labels_dir = '/content/coco_annotation'
dst_dir = '/content/yolo_dataset'
def add_coco_dataset(img_dir, labels_dir, dst_dir, train_size=0.8, val_size=0.2, test_size=1):
  random.seed(42)
  imgs = os.listdir(img_dir)
  random.shuffle(imgs)
  length = len(imgs)

  train_end = int(train_size * length)
  val_end = train_end + int(val_size * length)

  train_imgs = imgs[:train_end]
  val_imgs = imgs[train_end:val_end]
  test_imgs = imgs[val_end:]

  for img in [img for img in train_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/train'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/train'))
  print(f'{train_end} images have been move.')
  for img in [img for img in val_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/val'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/val'))
  print(f'{val_end-train_end} images have been move.')
  for img in [img for img in test_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/test'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/test'))
  print(f'{length-val_end} images have been move.')

add_coco_dataset(img_dir, labels_dir, dst_dir)

In [ ]:
# Load a model
model = YOLO('yolov8s-seg.yaml')  # build a new model from YAML
model = YOLO('yolov8s-seg.pt')  # load a pretrained model (recommended for training)
model.info()


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

(261, 11821056, 0, 42.860595200000006)

In [ ]:
# Train the model
model.train(data='/content/yolo_dataset/dataset.yaml', epochs=50, patience=10, imgsz=640, freeze=10, verbose=True, batch=-1, exist_ok=True,
      project='/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models', name='yolo_v8_coco_pt3.6/segment')

New https://pypi.org/project/ultralytics/8.2.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/content/yolo_dataset/dataset.yaml, epochs=50, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models, name=segment, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78dc5e5ebfd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v8_coco_pt3.6/segment'

## YOLO_DATASET VIDEO STREAM + COCO

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/video_yolo_dataset.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/coco-seg_5000samples.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
img_dir = '/content/coco-images'
labels_dir = '/content/coco_annotation'
dst_dir = '/content/yolo_dataset'
def add_coco_dataset(img_dir, labels_dir, dst_dir, train_size=0.7, val_size=0.2, test_size=0):
  imgs = os.listdir(img_dir)
  random.shuffle(imgs)
  length = len(imgs)

  train_end = int(train_size * length)
  val_end = train_end + int(val_size * length)

  train_imgs = imgs[:train_end]
  val_imgs = imgs[train_end:val_end]
  test_imgs = imgs[val_end:]

  for img in [img for img in train_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/train'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/train'))
  print(f'{train_end} images have been move.')
  for img in [img for img in val_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/val'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/val'))
  print(f'{val_end-train_end} images have been move.')
  for img in [img for img in test_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/test'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/test'))
  print(f'{length-val_end} images have been move.')

add_coco_dataset(img_dir, labels_dir, dst_dir)

3500 images have been move.
1000 images have been move.
500 images have been move.


In [ ]:
# Load a model
model = YOLO('yolov8s-seg.yaml')  # build a new model from YAML
model = YOLO('yolov8s-seg.pt')  # load a pretrained model (recommended for training)
model.info()


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

(261, 11821056, 0, 42.860595200000006)

In [ ]:
# Train the model
model.train(data='/content/yolo_dataset/dataset.yaml', epochs=50, patience=10, imgsz=640, freeze=10, verbose=True, batch=-1, exist_ok=True,
      project='/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models', name='yolo_v8_video_coco_pt3.6/segment')

New https://pypi.org/project/ultralytics/8.2.22 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/content/yolo_dataset/dataset.yaml, epochs=50, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models, name=segment, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78,
       79, 80])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cf78c27b850>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v8_video_coco_pt3.6/segment'

<IPython.core.display.Javascript object>

In [ ]:
drive.flush_and_unmount()